Voreinstellung

In [1]:
#r "BoSSSpad.dll"
using System;
using BoSSS.Foundation;
using BoSSS.Application.BoSSSpad;
using BoSSS.Solution;
using BoSSS.Solution.Tecplot;
using BoSSS.Foundation.IO;
using ilPSP.Tracing;
using BoSSS.Solution.Utils;
using ilPSP.LinSolvers;
using BoSSS.Solution.NSECommon;
using ilPSP.Connectors.Matlab;
using ilPSP;
using BoSSS.Foundation.Grid.Classic;
using ilPSP.Utils;
using BoSSS.Foundation.Grid.RefElements;
using System.Collections.Generic;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.XDG;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.AdvancedSolvers;
using System.Linq;
using BoSSS.Foundation.Grid.Aggregation;
using BoSSS.Platform;
using MPI.Wrappers;
using System.Diagnostics;
using System.IO;
using System.Collections;
using BoSSS.Foundation.Quadrature;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Gnuplot;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
Init();


In [2]:
#r "StokesHelical_Ak.dll"

In [3]:
using StokesHelical_Ak;

Voreinstellungen

In [4]:
string proj = "Helical_Centrifugal";

In [5]:
BoSSSshell.WorkflowMgm.Init(proj);

Project name is set to 'Helical_Centrifugal'.
Default Execution queue is chosen for the database.
Creating database '\\wsl.localhost\Ubuntu-22.04\home\akbari\lichtb-scratch\database\Helical_Centrifugal'.


In [6]:
BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();

In [7]:
GetDefaultQueue()

Username,sa90saxi
Password,<null>
ServerName,lcluster16.hrz.tu-darmstadt.de
SshClientExeToUse,<null>
PrivateKeyFilePath,P:\.ssh\id_ed25519
AdditionalBatchCommands,"[ #SBATCH -C avx512, #SBATCH --mem-per-cpu=3000 ]"
ExecutionTime,23:59:59
DeploymentBaseDirectoryAtRemote,/work/scratch/sa90saxi/deploy
RuntimeLocation,linux/amd64-openmpi
SlurmAccount,special00006
Email,akbari@fdy.tu-darmstadt.de


In [8]:
//ExecutionQueues

In [9]:
var myBatch = GetDefaultQueue();// ExecutionQueues[1];
myBatch

Username,sa90saxi
Password,<null>
ServerName,lcluster16.hrz.tu-darmstadt.de
SshClientExeToUse,<null>
PrivateKeyFilePath,P:\.ssh\id_ed25519
AdditionalBatchCommands,"[ #SBATCH -C avx512, #SBATCH --mem-per-cpu=3000 ]"
ExecutionTime,23:59:59
DeploymentBaseDirectoryAtRemote,/work/scratch/sa90saxi/deploy
RuntimeLocation,linux/amd64-openmpi
SlurmAccount,special00006
Email,akbari@fdy.tu-darmstadt.de


Projekt Name

In [10]:
var myDb = BoSSSshell.WorkflowMgm.DefaultDatabase;

## Creation of control files

In [11]:
int[] degrees = {4}; 
int[] noOfCellsRs = {64};
double rMin_ = 0;
int[] dtRefinings = {512};
string[] bdfOrders = {"BDF1"};
double[] MaxAmps = {5};
string[] grids = {"regular"};
bool[] iterative={false};

In [12]:
List<HelicalControl> Controls = new List<HelicalControl>();

In [13]:
Controls.Clear();

In [14]:
foreach(bool lin_inter_Solver in iterative){
    foreach (string grid in grids) {
        foreach (double MaxAmp in MaxAmps){
            foreach (int degree in degrees){     
                foreach (int noOfCellsR in noOfCellsRs){        
                    foreach (int dtRefining in dtRefinings){     
                        foreach (string bdfOrder in bdfOrders){

                        HelicalControl Ctrl = new HelicalControl();
                        Ctrl.DbPath = "@"+myDb; // _DbPath;
                        Ctrl.savetodb = Ctrl.DbPath != null;

                        int noOfCellsXi = noOfCellsR; // Aquidistnant
                        Ctrl.rMin = rMin_;
                        Ctrl.rMax = 1;
                        Ctrl.grid = grid;
                        Ctrl.Resolution_R = noOfCellsR;
                        Ctrl.Resolution_Xi = noOfCellsXi;
                        if (grid == "regular"){
                            Ctrl.GridFunc = delegate {
                                double[] xnodes = GenericBlas.Linspace(Ctrl.rMin, Ctrl.rMax, noOfCellsR + 1);
                                double[] ynodes = GenericBlas.Linspace(0, 2 * Math.PI, noOfCellsXi + 1);
                                GridCommons grd = Grid2D.Cartesian2DGrid(xnodes, ynodes, type: CellType.Square_Linear, periodicY: true);

                                grd.EdgeTagNames.Add(1, "Dirichlet_rmax");
                                grd.EdgeTagNames.Add(2, "Dirichlet_rmin");
                                grd.EdgeTagNames.Add(3, "Stuff");

                                grd.DefineEdgeTags(delegate (double[] _X) {
                                    var X = _X;
                                    double r, xi;
                                    r = X[0];
                                    xi = X[1];
                                    if(Math.Abs(r - Ctrl.rMax) < 1E-8)
                                        return 1;
                                    else if(Ctrl.rMin >= 1E-6 && Math.Abs(r - Ctrl.rMin) < 1E-8)
                                        return 2;
                                    else
                                        return 3;
                                });

                                return grd;
                            };
                        } else  {
                            Ctrl.GridFunc = delegate {
                                int numberOfSegments = 4;
                                // Initialisierung einer Liste für die Gitter
                                GridCommons[] grids = new GridCommons[numberOfSegments];
                                // Basisanzahl der Zellen im ersten Segment
                                int N = (noOfCellsR+1)/ numberOfSegments;
                                // Startpunkt für xNodes
                                double xStart = rMin_;
                                // Die Schrittweite für xNodes-Bereiche erhöhen
                                double xStep = (Ctrl.rMax / numberOfSegments) - xStart;

                                for(int i = 0; i < numberOfSegments; i++) {
                                    // Berechnung der Knotenanzahl für das aktuelle Segment
                                    int nodesInSegment = N * (int)Math.Pow(2, i) +1;

                                    // Bestimmung der x-Bereichsgrenzen für das aktuelle Segment
                                    double xEnd = xStart + xStep;

                                    // Erzeugung der x-Knoten und y-Knoten für das aktuelle Segment
                                    double[] xNodes = GenericBlas.Linspace(xStart, xEnd, nodesInSegment);
                                    double[] yNodes = GenericBlas.Linspace(0, 2 * Math.PI, nodesInSegment);

                                    // Erstellung des Gitters für das aktuelle Segment und Hinzufügen in die Liste
                                    grids[i] = Grid2D.Cartesian2DGrid(xNodes, yNodes, type: CellType.Square_Linear);
                                    // Aktualisierung des Startpunkts für den nächsten Durchlauf
                                    xStart = xEnd;
                                };

                                // Zusammenführen der Gitter aus der Liste
                                var grdJ = grids[0];
                                for(int i = 1; i < grids.Length; i++) {
                                    grdJ = GridCommons.MergeLogically(grids[i], grdJ);
                                }
                                // Versiegelung des zusammengeführten Gitters
                                GridCommons grd = GridCommons.Seal(grdJ, 4);

                                // Hinzufügen der Edge Tag Names und Definition der Edge Tags...
                                // (Füge hier den restlichen Teil deines Codes ein)

                                grd.EdgeTagNames.Add(1, "Dirichlet");
                                grd.EdgeTagNames.Add(2, "Stuff");

                                grd.DefineEdgeTags(delegate (double[] _X) {
                                    var X = _X;
                                    double r, xi;
                                    r = X[0];
                                    xi = X[1];
                                    if(Math.Abs(r - Ctrl.rMax) < 1E-8 || (Ctrl.rMin >= 1E-6 && Math.Abs(r - Ctrl.rMin) < 1E-8))
                                        return 1;
                                    else
                                        return 2;
                                });
                                return grd;
                            };
                        }       
                        double dt = 2 * Math.PI / (dtRefining*100);
                        Ctrl.dtMax = dt;
                        Ctrl.dtMin = dt;
                        if(bdfOrder == "BDF3") {
                            Ctrl.TimeSteppingScheme = TimeSteppingScheme.BDF3;
                        } else if(bdfOrder == "BDF1") {
                            Ctrl.TimeSteppingScheme = TimeSteppingScheme.ImplicitEuler;
                        } else {
                            throw new ArgumentException("Unsupported BDF scheme: " + bdfOrder);
                        }
                        Ctrl.NoOfTimesteps = dtRefining*10;

                        // Solver Properties
                        //=============
                        Ctrl.steady = false;
                        Ctrl.ExactResidual = false;
                        Ctrl.HagenPoisseulle = false;
                        // DG degree
                        // =========
                        Ctrl.dg_degree = degree;
                        Ctrl.SetDGdegree(degree);

                        double a = Globals.a;
                        double b = Globals.b;
                        double nu = Globals.nu;

                        // Initial Values
                        // ==============
                        Ctrl.AddInitialValue("Pressure", new Formula("(X) =>0"));
                        Ctrl.AddInitialValue("ur", new Formula("(X) => 0"));
                        Ctrl.AddInitialValue("ueta", new Formula($"(X) => 0"));
                        Ctrl.AddInitialValue("uxi", new Formula($"(X) => 0"));
                        // Boundary Conditions
                        // ==============
                        Ctrl.AddBoundaryValue("Dirichlet", "ur", new Formula("(X,t) =>  0", true));
                        Ctrl.AddBoundaryValue("Dirichlet", "ueta", new Formula($"(X,t) =>(X[0]/(Math.Sqrt({a * a} * X[0] * X[0] + {b * b} )))*{a}*{MaxAmp}* X[0]", true));
                        Ctrl.AddBoundaryValue("Dirichlet", "uxi", new Formula($"(X,t) => (X[0]/(Math.Sqrt({a * a} * X[0] * X[0] + {b * b} )))*{b}*{MaxAmp}", true));
                        Ctrl.AddBoundaryValue("Dirichlet", "Pressure", new Formula("(X,t) =>0", true));
                        // Solver Propertiees 
                        // ==============
                        Ctrl.maxAmpli = MaxAmp;
                        if(lin_inter_Solver == true){
                        Ctrl.LinearSolver = new BoSSS.Solution.AdvancedSolvers.OrthoMGSchwarzConfig() { ConvergenceCriterion = 1e-13 , CoarseKickIn = 1000000};
                        //Ctrl.LinearSolver = new BoSSS.Solution.AdvancedSolvers.OrthoMGSchwarzConfig() { ConvergenceCriterion = 1e-13 , TargetBlockSize =1000000};
                        }
                        
                        // Plotting
                        // ==============
                        Ctrl.ImmediatePlotPeriod =-1;
                        Ctrl.SuperSampling = 1;
                        Ctrl.TracingNamespaces= "*";
                        Controls.Add(Ctrl);
                        }
                    }
                }
            }
        }
    }
}

## Launch job

In [14]:
int [] NumberOfMPIProcs = {4};

In [15]:
foreach(int NumMPI in NumberOfMPIProcs ){
    foreach(HelicalControl C in Controls) {
        C.ProjectName = "Helical_HagenPoiseulle";
        C.SessionName = "BDF_" + C.GetBDFOrder()+ "_LSol_"+ C.LinearSolver.Shortname + "_Grid_"+ C.grid + "_rMin_" + C.rMin.ToString() + "_Mult_BSQ_"+ "_PRP_" + C.PressureReferencePoint + "_" + C.Resolution_R.ToString() + "_x_" +
                                    C.Resolution_R.ToString() + "_DGd_" + C.dg_degree+"_#MPI_"+ NumMPI +"_Amp_"+ C.maxAmpli.ToString() +"_dt_"+C.dtMax;
        var aJob   = new Job(C.SessionName,typeof(HelicalControl));
        aJob.SetControlObject(C);
        //aJob.NumberOfMPIProcs         = 1;
        //aJob.NumberOfThreads          = 4;
        aJob.UseComputeNodesExclusive = true;
        aJob.Activate(myBatch);    
    }
}

 ------------ MSHPC FailedOrCanceled; original Failed
Deployments so far (1): (Job token: 57946, FailedOrCanceled 'H_P-StokesHelical_Ak2024May10_110917.900037' @ MS HPC client  Default @DC3, @\\dc3\userspace\akbari\cluster\binaries, FailedOrCanceled);
Success: 0
job submit count: 1
Note: Job was deployed (1) number of times, all failed; RetryCount is 3, so try once more.
Hint: want to re-activate the job.
Job is FailedOrCanceled, but retry count is set to 3 and only 1 tries yet - trying once more...
Deploying job BDF_1_LSol_PARDISO_Grid_regular_rMin_0_Mult_one_PRP_True_64_x_64_DGd_4_#MPI_4_Amp_5_dt_0.0001227184630308513 ... 
Opening existing database '\\wsl.localhost\Ubuntu-22.04\home\akbari\lichtb-scratch\database\H_P'.
Opening existing database '\\dc3\userspace\akbari\Local\H_P'.
Set Database: { Session Count = 0; Grid Count = 2; Path = \\dc3\userspace\akbari\cluster\H_P }
Control object contains grid function. Trying to Serialize the grid...
Grid Edge Tags changed.
An equivalent gri

Deploying executables and additional files ...
Deployment directory: \\dc3\userspace\akbari\cluster\binaries\H_P-StokesHelical_Ak2024May10_131239.252749
copied 98 files.
   written file: control.obj
   copied 'win\amd64' runtime.
deployment finished.


In [ ]:
wmg.BlockUntilAllJobsTerminate(1.5*24*3600);

In [ ]:
wmg.AllJobs

In [ ]:
NUnit.Framework.Assert.Zero(wmg.AllJobs.Where(job => job.Status != JobStatus.FinishedSuccessful).Count(), "some non-succesful jobs found");